In [2]:
import pandas as pd

In [3]:
stats = pd.read_csv('mvp_stats.csv')

In [4]:
del stats['Unnamed: 0']

In [5]:
pd.isnull(stats).sum()

Player        0
Pos           0
Age           0
Tm            0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          58
3P            0
3PA           0
3P%        1886
2P            0
2PA           0
2P%         103
eFG%         58
FT            0
FTA           0
FT%         549
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
W           572
L           572
W/L%        572
GB            0
PS/G        572
PA/G        572
SRS         572
dtype: int64

In [6]:
stats = stats.fillna(0)

In [7]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

In [8]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']

In [9]:
train = stats[stats['Year'] < 2024]

In [10]:
test = stats[stats['Year'] == 2024]

In [11]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=0.1)

In [12]:
reg.fit(train[predictors], train['Share'])

Ridge(alpha=0.1)

In [13]:
predictions = reg.predict(test[predictors])

In [14]:
predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)

In [15]:
predictions

,Predictions
467,0.043355
468,0.070882
469,0.064254
470,0.075223
471,0.066077
...,...
14239,0.070671
14240,0.062644
14241,0.043851
14242,0.079572


In [16]:
combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)

In [17]:
combine


,Player,Share,Predictions
467,AJ Griffin,0.0,0.043355
468,Bogdan Bogdanović,0.0,0.070882
469,Bruno Fernando,0.0,0.064254
470,Clint Capela,0.0,0.075223
471,De'Andre Hunter,0.0,0.066077
...,...,...,...
14239,Marvin Bagley III,0.0,0.070671
14240,Patrick Baldwin Jr.,0.0,0.062644
14241,Richaun Holmes,0.0,0.043851
14242,Tristan Vukcevic,0.0,0.079572


In [18]:
combine.sort_values(by='Share', ascending=False).head(10)

,Player,Share,Predictions
3556,Nikola Jokić,0.935,0.222206
9654,Shai Gilgeous-Alexander,0.646,0.222172
3081,Luka Dončić,0.572,0.247123
7782,Giannis Antetokounmpo,0.194,0.271961
9370,Jalen Brunson,0.143,0.153325
950,Jayson Tatum,0.087,0.168258
8238,Anthony Edwards,0.018,0.141886
12011,Domantas Sabonis,0.003,0.159343
11082,Kevin Durant,0.001,0.159826
9643,Isaiah Joe,0.000,0.064032


In [19]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combine['Share'], combine['Predictions'])

0.00631199754710459

In [20]:
combine['Share'].value_counts()

Share
0.000    563
0.087      1
0.572      1
0.935      1
0.194      1
0.018      1
0.143      1
0.646      1
0.001      1
0.003      1
Name: count, dtype: int64

In [21]:
combine = combine.sort_values(by='Share', ascending=False)
combine['Rk'] = list(range(1, combine.shape[0] + 1))

In [22]:
combine.head(10)

,Player,Share,Predictions,Rk
3556,Nikola Jokić,0.935,0.222206,1
9654,Shai Gilgeous-Alexander,0.646,0.222172,2
3081,Luka Dončić,0.572,0.247123,3
7782,Giannis Antetokounmpo,0.194,0.271961,4
9370,Jalen Brunson,0.143,0.153325,5
950,Jayson Tatum,0.087,0.168258,6
8238,Anthony Edwards,0.018,0.141886,7
12011,Domantas Sabonis,0.003,0.159343,8
11082,Kevin Durant,0.001,0.159826,9
9643,Isaiah Joe,0.000,0.064032,10


In [23]:
combine = combine.sort_values(by='Predictions', ascending=False)
combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))

In [24]:
combine.head(10)

,Player,Share,Predictions,Rk,Predicted_Rk
7782,Giannis Antetokounmpo,0.194,0.271961,4,1
10612,Joel Embiid,0.000,0.256540,45,2
3081,Luka Dončić,0.572,0.247123,3,3
3556,Nikola Jokić,0.935,0.222206,1,4
9654,Shai Gilgeous-Alexander,0.646,0.222172,2,5
6420,LeBron James,0.000,0.189309,477,6
6408,Anthony Davis,0.000,0.189024,439,7
950,Jayson Tatum,0.087,0.168258,6,8
12502,Victor Wembanyama,0.000,0.163888,173,9
11082,Kevin Durant,0.001,0.159826,9,10


In [25]:
combine.sort_values(by='Share', ascending=False).head(10)

,Player,Share,Predictions,Rk,Predicted_Rk
3556,Nikola Jokić,0.935,0.222206,1,4
9654,Shai Gilgeous-Alexander,0.646,0.222172,2,5
3081,Luka Dončić,0.572,0.247123,3,3
7782,Giannis Antetokounmpo,0.194,0.271961,4,1
9370,Jalen Brunson,0.143,0.153325,5,14
950,Jayson Tatum,0.087,0.168258,6,8
8238,Anthony Edwards,0.018,0.141886,7,19
12011,Domantas Sabonis,0.003,0.159343,8,12
11082,Kevin Durant,0.001,0.159826,9,10
8869,Herbert Jones,0.000,0.052664,118,388


In [26]:
def find_ap(combine):
    actual = combine.sort_values(by='Share', ascending=False).head(5)
    predicted = combine.sort_values(by='Predictions', ascending=False)
    ps = []
    found = 0
    seen = 1
    for i, r in predicted.iterrows():
        if r['Player'] in actual['Player'].values:
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps)/len(ps)

In [27]:
find_ap(combine)

0.7147619047619048

In [28]:
years = list(range(1995, 2025))

In [29]:
aps = []
all_preds = []
for year in years[5:]:
    train = stats[stats['Year'] < year]
    test = stats[stats['Year'] == year]
    reg = Ridge(alpha=0.1)
    reg.fit(train[predictors], train['Share'])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
    combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)
    all_preds.append(combine)
    aps.append(find_ap(combine))

In [30]:
sum(aps)/len(aps)

0.7394186400549154

In [31]:
def add_ranks(combine):
    combine = combine.sort_values(by='Share', ascending=False)
    combine['Rk'] = list(range(1, combine.shape[0] + 1))
    combine = combine.sort_values(by='Predictions', ascending=False)
    combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))
    combine['Difference'] = combine['Rk'] - combine['Predicted_Rk']
    return combine

In [32]:
ranking = add_ranks(all_preds[1])
ranking[ranking['Rk'] < 6].sort_values(by='Difference', ascending=False)

,Player,Share,Predictions,Rk,Predicted_Rk,Difference
6047,Shaquille O'Neal,0.466,0.272036,3,1,2
11647,Chris Webber,0.420,0.157423,4,2,2
12128,Tim Duncan,0.569,0.137012,2,3,-1
7887,Kevin Garnett,0.122,0.115111,5,7,-2
10227,Allen Iverson,0.904,0.123538,1,5,-4


In [33]:
def backtest(stats, model, year, predictors):
    aps = []
    all_preds = []
    for year in years[5:]:
        train = stats[stats['Year'] < year]
        test = stats[stats['Year'] == year]
        model.fit(train[predictors], train['Share'])
        predictions = model.predict(test[predictors])
        predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
        combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)
        combine = add_ranks(combine)
        all_preds.append(combine)
        aps.append(find_ap(combine))
    return sum(aps)/len(aps), aps, pd.concat(all_preds)

In [34]:
mean_ap, aps, all_preds = backtest(stats,reg, years[5:], predictors)

In [35]:
mean_ap

0.7394186400549154

In [36]:
all_preds[all_preds['Rk']<=5].sort_values(by='Difference', ascending=False)

,Player,Share,Predictions,Rk,Predicted_Rk,Difference
8704,Anthony Davis,0.156,0.147815,5,1,4
6074,Shaquille O'Neal,0.106,0.214383,5,1,4
7725,Giannis Antetokounmpo,0.345,0.203472,4,1,3
2294,LeBron James,0.142,0.144979,5,2,3
12218,Tim Duncan,0.222,0.153712,4,1,3
...,...,...,...,...,...,...
10799,Steve Nash,0.739,0.060247,1,31,-30
11695,Peja Stojaković,0.228,0.034305,4,39,-35
1919,Joakim Noah,0.258,0.044454,4,40,-36
10781,Steve Nash,0.839,0.041184,1,41,-40


In [37]:
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(by=0, ascending=False)

,0,1
13,0.118759,eFG%
18,0.029277,DRB
29,0.028873,W/L%
17,0.016694,ORB
10,0.014796,2P
15,0.011408,FTA
12,0.010685,2P%
21,0.010172,STL
25,0.008678,PTS
22,0.007959,BLK


In [38]:
ratios = stats[['PTS', 'AST', 'STL', 'BLK', '3P', 'Year']].apply(lambda x: x/x.mean())

In [39]:
ratios

,PTS,AST,STL,BLK,3P,Year
0,1.177563,0.489219,0.777978,4.390800,0.000000,0.992408
1,1.177563,1.250227,1.400360,0.243933,1.608074,0.992408
2,0.218517,0.163073,0.155596,0.243933,0.000000,0.992408
3,0.315636,1.087154,1.089169,0.000000,0.160807,0.992408
4,0.097119,0.108715,0.000000,0.000000,0.321615,0.992408
...,...,...,...,...,...,...
14239,1.420360,0.597935,0.622382,1.707533,0.321615,1.006834
14240,0.534152,0.434862,0.777978,0.975733,1.286459,1.006834
14241,0.606991,0.326146,0.466787,1.219667,0.000000,1.006834
14242,1.031885,0.706650,0.777978,1.707533,1.608074,1.006834


In [40]:
stats[['PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']] = ratios[['PTS', 'AST', 'STL', 'BLK', '3P']]

In [41]:
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,W/L%,GB,PS/G,PA/G,SRS,PTS_T,AST_R,STL_R,BLK_R,3P_R
0,Andrew Lang,C,28,ATL,82,63,28.5,3.9,8.3,0.473,...,0.512,10.0,96.6,95.3,1.06,1.177563,0.489219,0.777978,4.390800,0.000000
1,Craig Ehlo,SG,33,ATL,49,0,23.8,3.9,8.6,0.453,...,0.512,10.0,96.6,95.3,1.06,1.177563,1.250227,1.400360,0.243933,1.608074
2,Doug Edwards,PF,24,ATL,38,0,5.6,0.6,1.3,0.458,...,0.512,10.0,96.6,95.3,1.06,0.218517,0.163073,0.155596,0.243933,0.000000
3,Ennis Whatley,PG,32,ATL,27,2,10.8,0.9,2.0,0.453,...,0.512,10.0,96.6,95.3,1.06,0.315636,1.087154,1.089169,0.000000,0.160807
4,Fred Vinson,SG,24,ATL,5,0,5.4,0.2,1.4,0.143,...,0.512,10.0,96.6,95.3,1.06,0.097119,0.108715,0.000000,0.000000,0.321615


In [42]:
predictors+=['PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']

In [43]:
mean_ap, aps, all_preds = backtest(stats, reg, years[5:], predictors)

In [44]:
mean_ap

0.7394186400549154

In [45]:
stats['NPos'] = stats['Pos'].astype('category').cat.codes

In [46]:
stats['NTm'] = stats['Tm'].astype('category').cat.codes

In [47]:
stats['NTm'].value_counts()    

NTm
7     498
27    495
17    495
8     489
14    487
0     485
33    483
12    482
31    478
15    477
1     476
13    475
18    475
11    475
26    474
9     473
29    472
5     472
10    471
24    467
30    465
28    465
34    464
19    461
36    449
16    397
20    286
25    274
2     220
32    209
23    198
6     169
3     157
21    143
4     121
35     88
37     47
22     32
Name: count, dtype: int64

In [48]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200, random_state=1, min_samples_split=10)

mean_ap, aps, all_preds = backtest(stats, rf, years[5:], predictors)

In [49]:
mean_ap

0.7249732097967392

In [50]:
mean_ap, aps, all_preds = backtest(stats, reg, years[5:], predictors)

In [51]:
mean_ap
# project inspired by dataquest tutorial 

0.7394186400549154

In [53]:
# Create the ratio features
ratios = stats[['PTS', 'AST', 'STL', 'BLK', '3P']].apply(lambda x: x/x.mean(), axis=0)

# Assign the new columns to the DataFrame
stats[['PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']] = ratios[['PTS', 'AST', 'STL', 'BLK', '3P']]

# Ensure these new columns are part of the training and test sets
train = stats[stats['Year'] < 2024]
test = stats[stats['Year'] == 2024]

# List of predictors should now include the new features
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 
              'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 
              'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS', 'PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']


In [54]:
train[predictors].isnull().sum()  # This should return 0 for all columns


Age      0
G        0
GS       0
MP       0
FG       0
FGA      0
FG%      0
3P       0
3PA      0
3P%      0
2P       0
2PA      0
2P%      0
eFG%     0
FT       0
FTA      0
FT%      0
ORB      0
DRB      0
TRB      0
AST      0
STL      0
BLK      0
TOV      0
PF       0
PTS      0
Year     0
W        0
L        0
W/L%     0
GB       0
PS/G     0
PA/G     0
SRS      0
PTS_T    0
AST_R    0
STL_R    0
BLK_R    0
3P_R     0
dtype: int64

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

ridge_params = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10, 100]
}

ridge = Ridge()



In [56]:
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, scoring='neg_mean_squared_error')
ridge_grid.fit(train[predictors], train['Share'])


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1.0, 10, 100]},
             scoring='neg_mean_squared_error')

In [57]:
best_ridge_params = ridge_grid.best_params_
best_ridge_score = -ridge_grid.best_score_

In [58]:
best_ridge_score

0.002741741659337381

In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf_params = {
    'n_estimators': [100, 200, 300],         
    'max_depth': [10, 20, 30, None],         
    'min_samples_split': [2, 5, 10],          
    'max_features': ['auto', 'sqrt', 'log2']
}

rf = RandomForestRegressor(random_state=1)

rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_grid.fit(train[predictors], train['Share'])

best_rf_params = rf_grid.best_params_
best_rf_score = -rf_grid.best_score_

print(f"Best RandomForest Parameters: {best_rf_params}")
print(f"Best RandomForest Cross-Validated Score (MSE): {best_rf_score}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
162 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.1

Best RandomForest Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 300}
Best RandomForest Cross-Validated Score (MSE): 0.0013708236333959864


In [60]:
final_rf = RandomForestRegressor(max_depth=20, max_features='sqrt', min_samples_split=2, n_estimators=300, random_state=1)
final_rf.fit(train[predictors], train['Share'])


RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=300,
                      random_state=1)

In [61]:
predictions = final_rf.predict(test[predictors])


In [62]:
predictions_df = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
combine = pd.concat([test[['Player', 'Share']], predictions_df], axis=1)

In [74]:
mse = mean_squared_error(combine['Share'], combine['Predictions'])
print(f"Test MSE: {mse}")

Test MSE: 0.0021352592587755032


In [75]:
combine = combine.sort_values(by='Predictions', ascending=False)
combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))
print(combine.head(3))

                       Player  Share  Predictions  Predicted_Rk
3556             Nikola Jokić  0.935     0.503030             1
7782    Giannis Antetokounmpo  0.194     0.427783             2
9654  Shai Gilgeous-Alexander  0.646     0.420570             3


In [76]:
# Filter for the year 2024
actual_2024 = stats[stats['Year'] == 2024]

top_3_actual = actual_2024.sort_values(by='Share', ascending=False).head(3)

top_3_actual['Actual_Rank'] = list(range(1, top_3_actual.shape[0] + 1))

print(top_3_actual[['Player', 'Share', 'Actual_Rank']])


                       Player  Share  Actual_Rank
3556             Nikola Jokić  0.935            1
9654  Shai Gilgeous-Alexander  0.646            2
3081              Luka Dončić  0.572            3


In [77]:
combine = combine.sort_values(by='Predictions', ascending=False)

combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))

print(combine[['Player', 'Predicted_Rk']].head(3))


                       Player  Predicted_Rk
3556             Nikola Jokić             1
7782    Giannis Antetokounmpo             2
9654  Shai Gilgeous-Alexander             3


In [78]:
comparison = pd.merge(top_3_actual[['Player', 'Actual_Rank']], combine[['Player', 'Predicted_Rk']], on='Player', how='inner')

comparison.columns = ['Player', 'Actual Rank', 'Predicted Rank']

print(comparison)


                    Player  Actual Rank  Predicted Rank
0             Nikola Jokić            1               1
1  Shai Gilgeous-Alexander            2               3
2              Luka Dončić            3               5


In [79]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(comparison['Actual Rank'], comparison['Predicted Rank'])

print(f"Mean Absolute Error of Ranks: {mae}")


Mean Absolute Error of Ranks: 1.0


In [83]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

xgb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'alpha': [0, 0.01, 0.1],  
    'lambda': [1, 10, 100]    
}

xgb = XGBRegressor(random_state=1)

xgb_grid = GridSearchCV(xgb, xgb_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
xgb_grid.fit(train[predictors], train['Share'])

best_xgb_params = xgb_grid.best_params_
best_xgb_score = -xgb_grid.best_score_

print(f"Best XGBoost Parameters with Regularization: {best_xgb_params}")
print(f"Best XGBoost Cross-Validated Score with Regularization (MSE): {best_xgb_score}")

final_xgb = XGBRegressor(**best_xgb_params, random_state=1)
final_xgb.fit(train[predictors], train['Share'])

xgb_predictions = final_xgb.predict(test[predictors])

xgb_combine = pd.DataFrame(xgb_predictions, columns=['Predictions'], index=test.index)
xgb_combine = pd.concat([test[['Player', 'Share']], xgb_combine], axis=1)

xgb_mse = mean_squared_error(xgb_combine['Share'], xgb_combine['Predictions'])
print(f"XGBoost Test MSE with Regularization: {xgb_mse}")

xgb_combine = xgb_combine.sort_values(by='Predictions', ascending=False)
xgb_combine['Predicted_Rk'] = list(range(1, xgb_combine.shape[0] + 1))
print(xgb_combine.head(10))


Best XGBoost Parameters with Regularization: {'alpha': 0.1, 'colsample_bytree': 0.8, 'lambda': 10, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
Best XGBoost Cross-Validated Score with Regularization (MSE): 0.0012476580755143502
XGBoost Test MSE with Regularization: 0.0038483854450051883
                        Player  Share  Predictions  Predicted_Rk
3556              Nikola Jokić  0.935     0.891975             1
7782     Giannis Antetokounmpo  0.194     0.780749             2
9654   Shai Gilgeous-Alexander  0.646     0.754817             3
3081               Luka Dončić  0.572     0.557129             4
6408             Anthony Davis  0.000     0.546216             5
10612              Joel Embiid  0.000     0.529252             6
6420              LeBron James  0.000     0.403183             7
11082             Kevin Durant  0.001     0.384580             8
8881           Zion Williamson  0.000     0.360644             9
9370             Jalen Brunson

In [84]:
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=10, shuffle=True, random_state=1)

cv_results = cross_val_score(final_xgb, train[predictors], train['Share'], cv=kf, scoring='neg_mean_squared_error')
mean_cv_score = -cv_results.mean()

print(f"Mean Cross-Validated Score (MSE) Across Different Splits: {mean_cv_score}")


Mean Cross-Validated Score (MSE) Across Different Splits: 0.0010893193248410322


In [85]:
rf_results = cross_val_score(rf, train[predictors], train['Share'], cv=kf, scoring='neg_mean_squared_error')
mean_rf_score = -rf_results.mean()

print(f"Mean Cross-Validated Score for RandomForest (MSE): {mean_rf_score}")


Mean Cross-Validated Score for RandomForest (MSE): 0.0011896805433375048


In [86]:
ridge_results = cross_val_score(ridge, train[predictors], train['Share'], cv=kf, scoring='neg_mean_squared_error')
mean_ridge_score = -ridge_results.mean()

print(f"Mean Cross-Validated Score for Ridge Regression (MSE): {mean_ridge_score}")


Mean Cross-Validated Score for Ridge Regression (MSE): 0.0026906023974882683


In [87]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf_params = {
    'n_estimators': [200, 300, 400],             
    'max_depth': [15, 20, 25],                   
    'min_samples_split': [2, 5, 10],             
    'min_samples_leaf': [1, 2, 4],               
    'max_features': ['auto', 'sqrt', 'log2'],    
    'bootstrap': [True, False]                  
}

rf = RandomForestRegressor(random_state=1)

rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_grid.fit(train[predictors], train['Share'])

best_rf_params = rf_grid.best_params_
best_rf_score = -rf_grid.best_score_

print(f"Best RandomForest Parameters: {best_rf_params}")
print(f"Best RandomForest Cross-Validated Score (MSE): {best_rf_score}")

final_rf = RandomForestRegressor(**best_rf_params, random_state=1)
final_rf.fit(train[predictors], train['Share'])

rf_predictions = final_rf.predict(test[predictors])

rf_combine = pd.DataFrame(rf_predictions, columns=['Predictions'], index=test.index)
rf_combine = pd.concat([test[['Player', 'Share']], rf_combine], axis=1)

rf_mse = mean_squared_error(rf_combine['Share'], rf_combine['Predictions'])
print(f"RandomForest Test MSE with Best Parameters: {rf_mse}")

rf_combine = rf_combine.sort_values(by='Predictions', ascending=False)
rf_combine['Predicted_Rk'] = list(range(1, rf_combine.shape[0] + 1))
print(rf_combine.head(10))


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
810 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
740 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.

Best RandomForest Parameters: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best RandomForest Cross-Validated Score (MSE): 0.001319165608035176
RandomForest Test MSE with Best Parameters: 0.0020327765901364783
                        Player  Share  Predictions  Predicted_Rk
3556              Nikola Jokić  0.935     0.549538             1
7782     Giannis Antetokounmpo  0.194     0.505362             2
9654   Shai Gilgeous-Alexander  0.646     0.500495             3
10612              Joel Embiid  0.000     0.446068             4
3081               Luka Dončić  0.572     0.400905             5
6420              LeBron James  0.000     0.290295             6
6408             Anthony Davis  0.000     0.278461             7
11082             Kevin Durant  0.001     0.277256             8
9370             Jalen Brunson  0.143     0.251304             9
11074             Devin Booker  0.000     0.195767      

In [88]:
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=10, shuffle=True, random_state=1)

cv_results_rf = cross_val_score(final_rf, train[predictors], train['Share'], cv=kf, scoring='neg_mean_squared_error')
mean_cv_score_rf = -cv_results_rf.mean()

print(f"Mean Cross-Validated Score for RandomForest (MSE) Across Different Splits: {mean_cv_score_rf}")


Mean Cross-Validated Score for RandomForest (MSE) Across Different Splits: 0.001178667412859458


In [89]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(rf_combine['Share'], rf_combine['Predictions'])

print(f"Mean Absolute Error (MAE): {mae}")


Mean Absolute Error (MAE): 0.011521409426507086
